In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras import layers, initializers

import numpy as np
import os
import random
import matplotlib.pyplot as plt

import importlib
import utils.ml_utils as ml_utils
importlib.reload(ml_utils)

<module 'utils.ml_utils' from '/home/darylchang/ml-experiments/utils/ml_utils.py'>

In [4]:
ml_utils.init_env()

In [ ]:
# Constants
IMG_SIZE = 128 # All images in the dataset will be resized to this size
BATCH_SIZE = 32 # Batch size

# Read in training and validation data
train, validation = ml_utils.load_batched_and_resized_dataset(
    dataset_name='cats_vs_dogs',
    batch_size=BATCH_SIZE,
    img_size=IMG_SIZE
)

In [ ]:
import tensorflow.keras as keras
from ray.tune import track
from ray import tune
import ray
from ray.tune.utils import pin_in_object_store, get_pinned_object

ray.shutdown()
ray.init(memory=8*10**9, object_store_memory=6*10**9)

In [ ]:
class TuneReporterCallback(keras.callbacks.Callback):
    """Tune Callback for Keras.
    
    The callback is invoked every epoch.
    """

    def __init__(self, logs={}):
        self.iteration = 0
        super(TuneReporterCallback, self).__init__()

    def on_epoch_end(self, batch, logs={}):
        self.iteration += 1
        track.log(keras_info=logs, mean_accuracy=logs.get("accuracy"), mean_loss=logs.get("loss"))

In [ ]:
train_x = np.concatenate([x for x, y in tfds.as_numpy(train)], axis=0)

In [ ]:
train_y = np.concatenate([y for x, y in tfds.as_numpy(train)], axis=0)

In [ ]:
train_x.shape

In [ ]:
train_y.shape

In [ ]:
validation_x = np.concatenate([x for x, y in tfds.as_numpy(validation)], axis=0)
validation_y = np.concatenate([y for x, y in tfds.as_numpy(validation)], axis=0)

In [ ]:
train_x_id = pin_in_object_store(train_x)
train_y_id = pin_in_object_store(train_y)
validation_x_id = pin_in_object_store(validation_x)
validation_y_id = pin_in_object_store(validation_y)

In [ ]:
def train_model(config):
    tr = TuneReporterCallback()
    train_x, train_y = get_pinned_object(train_x_id), get_pinned_object(train_y_id)
    validation_x, validation_y = get_pinned_object(validation_x_id), get_pinned_object(validation_y_id)
    model = ml_utils.build_model(optimizer=keras.optimizers.SGD(lr=config['lr']))
    model.fit(
        train_x,
        train_y,
        validation_data=(validation_x, validation_y),
        epochs=1,
        verbose=1,
        callbacks=[tr],
    )

In [2]:
analysis = tune.run(
    train_model, 
    verbose=1,
    config={"lr": tune.loguniform(0.001, 0.1)},
    num_samples=3,
    resources_per_trial={"gpu": 1}
)

NameError: name 'tune' is not defined